# Анализ приложений в AppStore и GooglePlay
---
В данном проекте мы притворяемся аналитоком в компании, которая занимается созданием бесплатных приложений. Мы поможем ей определить какими должны быть эти приложения, чтобы они могли приносить наибольшую прибыль.

Для этого проанализируем уже существующие приложения в **AppStore** и **GooglePlay**.
    
#### Наборы данных

В этом проект мы воспользуемся уже готовыми наборами данных:
 - Для [AppStore](https://www.kaggle.com/lava18/google-play-store-apps) от августа 2018
 - Для [GooglePlay](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) от июля 2017
    
Создадим функцию `save_as_list` для преобразования этих данных во вложенные списки    

In [1]:
def save_as_list(dataset):
    opened_dataset = open(dataset)
    from csv import reader 
    read_dataset = reader(opened_dataset)
    dataset_as_list = list(read_dataset)
    return dataset_as_list

Преобразуем наборы данных

In [2]:
appstore_data = save_as_list('AppleStore.csv')
googleplay_data = save_as_list('googleplaystore.csv')

Задаём исследовательскую функцию `explore_data`, для первичного анализа

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row, '\n')
    if rows_and_columns:
        print('Количесво строк: ', len(dataset))
        print('Количество столбцов: ', len(dataset[0]))    

Применим функцию `explore_data` к нашим данным

In [4]:
explore_data(appstore_data[1:], 0, 5, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'] 

['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1'] 

Количесво строк:  7197
Количество столбцов:  16


In [5]:
explore_data(googleplay_data[1:], 0, 5, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'] 

Количесво строк:  10841
Количество столбцов:  13


Отдельно выведем названия всех столбцов для обоих датасетов

In [6]:
print(appstore_data[0], '\n')
print(googleplay_data[0])

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


#### Обработка данных
Наша компания занимается только **бесплатными** приложения, поэтому мы уберём все платные из наших данных. Также мы оставим только приложения на **английском языке**, так как мы нацелены на англоговорящую аудиторию.

Но предже всего найдём ошибки в данных.

In [7]:
print(googleplay_data[10473], len(googleplay_data[10473]))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 12


В данной строке не хватает одного значения, мы её удалим.

In [8]:
del googleplay_data[10473] #Закомментили после единичного удаления, чтобы случайно не удалить нужные данные

Также необходимо проверить есть в списках повторяющиеся приложения. Для этого создадим функцию `duplicate_and_unique_app`.

In [9]:
def duplicate_and_unique_app(data, column):
    unique_app=[]
    duplicate_app=[]
    for row in data:
        if row[column] in unique_app:
            duplicate_app.append(row[column])
        else:
            unique_app.append(row[column])
    print('Количесвтво повторяющихся строк:', len(duplicate_app))
    return unique_app, duplicate_app

Проверим данные из **GooglePlay**:

In [10]:
unique_google, duplicate_google = duplicate_and_unique_app(googleplay_data, 0)

Количесвтво повторяющихся строк: 1181


Из **AppStore**:

In [11]:
unique_apple, duplicate_apple = duplicate_and_unique_app(appstore_data, 1)

Количесвтво повторяющихся строк: 2


Давайте подробнее рассмотрим какое-то одно повторяющееся приложение. В качестве примера возьмём самое частотное.

In [12]:
freq_dupl = {}
for app in duplicate_google:
    if app in freq_dupl:
        freq_dupl[app] += 1
    else:
        freq_dupl[app] = 1
max_freq_value = max(freq_dupl, key=lambda x: freq_dupl[x])        
print(max_freq_value, '\n')
for app in googleplay_data:
    if app[0] == max_freq_value:
        print(app, '\n')

ROBLOX 

['ROBLOX', 'GAME', '4.5', '4447388', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '2.347.225742', '4.1 and up'] 

['ROBLOX', 'GAME', '4.5', '4447346', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '2.347.225742', '4.1 and up'] 

['ROBLOX', 'GAME', '4.5', '4448791', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '2.347.225742', '4.1 and up'] 

['ROBLOX', 'GAME', '4.5', '4449882', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '2.347.225742', '4.1 and up'] 

['ROBLOX', 'GAME', '4.5', '4449910', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '2.347.225742', '4.1 and up'] 

['ROBLOX', 'FAMILY', '4.5', '4449910', '67M', '100,000,000+', 'Free', '0', 'Everyone 10+', 'Adventure;Action & Adventure', 'July 31, 2018', '

Самый частотным оказалось приложение **"Roblox"**. Как можно заметить, единственное различие в записях - разное количество отзывов.

Можно предположить, что запись с наибольим количество отзывов самая актуальная. Соотвественно можно воспользовать этим для удаления повторяющихся записей.
> Критерий отбора - количество отзывов

Создадим функцию `app_reviews` для создания пар приложение - кол-во отзывов, исключая повторы

In [13]:
def app_reviews(dataset, column_1, column_2):
    max_reviews = {}
    for row in dataset[1:]:
        name = row[column_1]
        qty_reviews = float(row[column_2])
        if name not in max_reviews:
            max_reviews[name] = qty_reviews
        else:
            if max_reviews[name] < qty_reviews:
                max_reviews[name] = qty_reviews
    return max_reviews  

Применим `apps_reviews` к обоим наборам данных и произведём небольшую проверку.

In [14]:
apple_max_reviews = app_reviews(appstore_data, 1, 5)
google_max_reviews = app_reviews(googleplay_data, 0, 3)
print(len(apple_max_reviews), len(google_max_reviews))
print(google_max_reviews['ROBLOX'])

7195 9659
4450890.0


Теперь применим полученные словари `apple_max_reviews` и `google_max_reviews` для того, чтобы избавиться от повторяющихся данных в наших датасетах.

In [15]:
def clean_apps(dataset, reviews_dictionary, column_1, column_2):
    cleaned_apps_list = []
    added_app = []
    for row in dataset[1:]:
        name = row[column_1]
        qty_reviews = float(row[column_2])
        if (qty_reviews == reviews_dictionary[name]) and (name not in added_app):
            cleaned_apps_list.append(row)
            added_app.append(name)
    return cleaned_apps_list

apple_cleaned = clean_apps(appstore_data, apple_max_reviews, 1, 5)
google_cleaned = clean_apps(googleplay_data, google_max_reviews, 0, 3)

print(len(apple_cleaned), len(google_cleaned))

7195 9659


После того как мы избавились от повторяющихся данных можно приступить к отсеяванию приложений не на английском языке.

Для этого воспользуемся кодировкой [ASCII](https://ru.wikipedia.org/wiki/ASCII), в которой каждому символу соотвествует определнное число.

Согласно ей все латинские буквы, цифры, а также знаки препинания и некоторые другие символы (%, $ и тд) соотвествуют числам от 0 до 127 включительно.

![image](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/ASCII_Code_Chart.svg/2560px-ASCII_Code_Chart.svg.png)

Вопользуемся этим, а также встроенной функцией `ord()`

In [16]:
def non_english_app(row, name_column):
    non_eng_symbols = 0
    for symbol in row[name_column]:
        if ord(symbol) > 127:
            non_eng_symbols += 1
        #print(non_eng_symbols)
    return non_eng_symbols <= 3

            
test_list = [['爱奇艺PPS -《欢乐颂2》电视剧热播'],
            ['爱奇艺PPS -'],
            ['Docs To Go™ Free Office Suite'],
            ['Instachat 😜']]    
non_english_app(test_list[1], 0)

True

Применим только заданную функцию `non_english_app`, чтобы получить списки только с англ. приложениями. Также получим списки с оставшимися приложения для самопроверки.

In [17]:
def keep_eng_only(dataset, name_column):
    only_eng_list = []
    only_non_eng_list = []
    for row in dataset:
        if non_english_app(row, name_column) == True:
            only_eng_list.append(row)
        else:
            only_non_eng_list.append(row)
    return only_eng_list, only_non_eng_list

apple_eng_only, apple_non_eng_list = keep_eng_only(apple_cleaned, 1)
google_eng_only, google_non_eng_list = keep_eng_only(google_cleaned, 0)

print(len(apple_eng_only), len(google_eng_only))
print(apple_non_eng_list[:5])

6181 9614
[['445375097', '爱奇艺PPS -《欢乐颂2》电视剧热播', '224617472', 'USD', '0.0', '14844', '0', '4.0', '0.0', '6.3.3', '17+', 'Entertainment', '38', '5', '3', '1'], ['405667771', '聚力视频HD-人民的名义,跨界歌王全网热播', '90725376', 'USD', '0.0', '7446', '8', '4.0', '4.5', '5.0.8', '12+', 'Entertainment', '24', '4', '1', '1'], ['336141475', '优酷视频', '204959744', 'USD', '0.0', '4885', '0', '3.5', '0.0', '6.7.0', '12+', 'Entertainment', '38', '0', '2', '1'], ['425349261', '网易新闻 - 精选好内容，算出你的兴趣', '133134336', 'USD', '0.0', '4263', '6', '4.5', '1.0', '23.2', '17+', 'News', '37', '4', '2', '1'], ['387682726', '淘宝 - 随时随地，想淘就淘', '309673984', 'USD', '0.0', '3801', '6', '4.0', '4.0', '6.7.2', '4+', 'Shopping', '37', '1', '1', '1']]


Тепеь у нас есть два списка только с приложениями на английском языке для обоих маркетов: `apple_eng_only` и `google_eng_only`.

Теперь оставим только бесплатные приложения, с помощью нами заданной функции `free_only`.

In [18]:
def free_only(dataset, price_column, free_price_value):
    free_only_list = []
    for row in dataset:
        if row[price_column] == free_price_value:
            free_only_list.append(row)
    return free_only_list

apple_free_apps = free_only(apple_eng_only, 4, '0.0')
google_free_apps = free_only(google_eng_only, 6, 'Free')

print(len(apple_free_apps), len(google_free_apps))

3220 8863


В результате всех процедур по подготовке данных у нас остались:
 - 3220 релевантных записи для **AppStore**
 - 8863 релевантных записи для **GooglePlay**
 
#### Анализ данных 

Наша компания имеет следущую стратегию по реализации приложений:
- Разработать простейшую версию приложения для **Android** и добавить его в **GooglePlay**
- Развивать его дальше при положительных отзывах пользователей
- После полугода, в течение которых приложение бы приносило прибыль, разработать версию приложения для **iOS** и добавить его в **AppStore**

Давайте надйдём в каких категориях самые популярные приложения. Создадим функию `pop_genres` и применем к подготовленным выше датасетам.

In [19]:
def pop_genres(dataset, genre_column, reviews_column):
    pop_genres_dict = {}
    pop_genres_reviews_count = {}
    for row in dataset:
        genre = row[genre_column]
        reviews = float(row[reviews_column])
        if genre not in pop_genres_dict:
            pop_genres_dict[genre] = 1
        else:
            pop_genres_dict[genre] += 1
            
        if genre not in pop_genres_reviews_count:
            pop_genres_reviews_count[genre] = reviews
        else:
            pop_genres_reviews_count[genre] += reviews     
    return pop_genres_dict, pop_genres_reviews_count

freq_genre_apple, reviews_count_apple = pop_genres(apple_free_apps, 11, 5)
freq_category_google, reviews_count_category_google = pop_genres(google_free_apps, 1, 3)

freq_genre_google, reviews_count_genre_google = pop_genres(google_free_apps, 9, 3)

print('AppStore genres', '\n')
for note in freq_genre_apple:
    print(note, ': ', freq_genre_apple[note], '\n')

#print('GooglePlay genres', '\n')
#for note in freq_genre_google:
#    print(note, ': ', freq_genre_google[note], '\n')

AppStore genres 

Social Networking :  106 

Photo & Video :  160 

Games :  1872 

Music :  66 

Reference :  18 

Health & Fitness :  65 

Weather :  28 

Utilities :  81 

Travel :  40 

Shopping :  84 

News :  43 

Navigation :  6 

Lifestyle :  51 

Entertainment :  254 

Food & Drink :  26 

Sports :  69 

Book :  14 

Finance :  36 

Education :  118 

Productivity :  56 

Business :  17 

Catalogs :  4 

Medical :  6 



Мы получили количества приложений каждого жанра для обоих площадок, а также количество отзывов на приложения каждого жанра. Причём для GooglePlay мы получили данные по двум столбцам: **"Category"** и **"Genre"**.

Чтобы проще было воспринимать эти данные, давайте приведём их к более удобному виду: отсоритруем и переведём в доли. Для этого задаёс функцию `sort_freq_table`.

In [20]:
def sort_freq_table(freq_table):
    table_value_key = []
    table_sum = 0 #Задаём эту переменную сугубо для случая с кол-вом обзоров, сумма приложений нам и так известна
    for key in freq_table:
        table_sum += freq_table[key]
    for key in freq_table:
        as_tuple = ((freq_table[key] / table_sum), key)
        table_value_key.append(as_tuple)
    sort_freq_table = sorted(table_value_key, reverse=True)    
    return sort_freq_table

Применем заданную функцию поочередно ко всем нашим данным. Начнём с **AppStore**, столбец **"prime_genre"**.

In [21]:
sort_freq_apple = sort_freq_table(freq_genre_apple)

print('Топ 5 популярных жанров:')
for row in sort_freq_apple[:5]:
    print(row[1], ': {:.2%}'.format(row[0]))

Топ 5 популярных жанров:
Games : 58.14%
Entertainment : 7.89%
Photo & Video : 4.97%
Education : 3.66%
Social Networking : 3.29%


In [22]:
sort_freq_apple_reviews = sort_freq_table(reviews_count_apple)

print('Топ 5 жанров с наибольшим кол-вом отзывов:')
for row in sort_freq_apple_reviews[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 жанров с наибольшим кол-вом отзывов:
Games 53.39%
Social Networking 9.48%
Photo & Video 5.69%
Music 4.73%
Entertainment 4.46%


Теперь для **GooglePlay**, для столбца **"Category"**.

In [23]:
sort_freq_cat_google = sort_freq_table(freq_category_google)

print('Топ 5 по категориям:')
for row in sort_freq_cat_google[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 по категориям:
FAMILY 18.90%
GAME 9.73%
TOOLS 8.46%
BUSINESS 4.59%
LIFESTYLE 3.90%


In [24]:
sort_freq_cat_review_google = sort_freq_table(reviews_count_category_google)

print('Топ 5 по категориям и кол-ву отзывов:')
for row in sort_freq_cat_review_google[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 по категориям и кол-ву отзывов:
GAME 28.23%
COMMUNICATION 13.69%
TOOLS 10.99%
SOCIAL 10.92%
FAMILY 9.09%


И последнее - **GooglePlay**, столбец **"Genres"**.

In [25]:
sort_freq_genre_google = sort_freq_table(freq_genre_google)

print('Топ 5 по жанрам:')
for row in sort_freq_genre_google[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 по жанрам:
Tools 8.45%
Entertainment 6.07%
Education 5.35%
Business 4.59%
Productivity 3.89%


In [26]:
sort_freq_genre_review_google = sort_freq_table(reviews_count_genre_google)

print('Топ 5 по жанрам и кол-ву отзывов:')
for row in sort_freq_genre_review_google[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 по жанрам и кол-ву отзывов:
Communication 13.69%
Tools 10.97%
Social 10.92%
Action 7.18%
Casual 6.26%


По полученным данным для **AppStore** сразу заметно, что с более чем пятикратным преимуществом лидирует жанр **"Games"** как по количеству самих приложений, так и по количеству отзывов. **58.14%** и **53.36%** соответственно.

С данными по **GogglePlay** несколько сложнее. Явный лидер по категориям - **"FAMILY"** с **18,9%**. В эту категорию, на мой взгляд могут попадать совершенно разные приложения. По кол-ву отзывов категория **"FAMILY"** на пятом месте с **9,09%**.

На втором месте идёт категория **"GAMES"** с **9,73%**, она же лидирует по кол-ву отзывов с **28.23%**.

Также с играми можно соотнести некотоhые данные по столбцу **"Genres"**. В топ 5 по отзывам там присутствуют **"Casual"** и **"Action"**, что является играми. К категории **"FAMILY"** можно причислить любой жанр из топ-5.

Можно сделать предварительный вывод о необходимости разрабатывать игру, подходящую для семейного использользования.

Чтобый получить дополнитлеьные сведения, давайте обратимся к столбцу **"Installs"** - кол-во установок. В датасете количество задано не конкретным число а интервалом, например **100,000+**. Точного кол-ва мы всё равно не узнаем, поэтому давайте примем за него нижнюю границу интервала => **100,000+ = 100000**.

Модернизируем нашу функцию `pop_genres` в `pop_genres_2`.

In [27]:
def pop_genres_2(dataset, genre_column, installs_column):
    #pop_genres_dict = {}
    pop_genres_installs_count = {}
    for row in dataset:
        genre = row[genre_column]
        temp_installs = row[installs_column].replace(',', '')
        installs = float(temp_installs.replace('+', ''))
           
        if genre not in pop_genres_installs_count:
            pop_genres_installs_count[genre] = installs
        else:
            pop_genres_installs_count[genre] += installs     
    return pop_genres_installs_count

Теперь примененим её на датасет вместе с сортирующей функциуй `sort_freq_table`.

In [28]:
installs_count_category_google = pop_genres_2(google_free_apps, 1, 5)

sort_installs_count_category_google = sort_freq_table(installs_count_category_google)

print('Топ 5 категорий с наибольшим числом установок:')
for row in sort_installs_count_category_google[:5]:
    print(row[1], '{:.2%}'.format(row[0]))

Топ 5 категорий с наибольшим числом установок:
GAME 17.86%
COMMUNICATION 14.67%
TOOLS 10.77%
FAMILY 8.23%
PRODUCTIVITY 7.70%


По кол-ву установок явно лидирует категория **"Games"** с **17,86%**.

Соотвественно у нас получанется следующее:
 - Более половины всех отзывов в **AppStore** относятся к играм (**53.36%**)
 - Более половины всех приложений в **AppStore** - игры (**58.14%**)
 - Больше всех установленных приложений в **GooglePlay** - игры (**17.86%**)
 - Больше всего отзывов на  приложения в **GooglePlay** - на игры (**28.23%**)
 
Кажется, что сделать игру - это наилучший вариант. Больше всего  установок и отзывов могут наводить на эту мысль. Но такие результаты обусловлены большим кол-вом самих игр в магазинах. Давайте посмотрим сколько отзывов или установок приходится на одно приложение по жанрам.

In [29]:
def avg_reviews(genre_data, reviews_data):
    avg_table = {}
    presort_avg_table = []
    for key in genre_data:
        avg_table[key] = reviews_data[key] / genre_data[key]
    for key in avg_table:
        as_tuple = (avg_table[key], key)
        presort_avg_table.append(as_tuple)
    sort_avg_table = sorted(presort_avg_table, reverse=True)    
    return sort_avg_table    

avg_reviews_apple = avg_reviews(freq_genre_apple, reviews_count_apple)

for row in avg_reviews_apple:
    print(row[1], ':{:.6}'.format(row[0]))

Navigation :86090.3
Reference :74942.1
Social Networking :71548.3
Music :57326.5
Weather :52279.9
Book :39758.5
Food & Drink :33333.9
Finance :31467.9
Photo & Video :28441.5
Travel :28243.8
Shopping :26919.7
Health & Fitness :23298.0
Sports :23008.9
Games :22812.9
News :21248.0
Productivity :21028.4
Utilities :18684.5
Lifestyle :16485.8
Entertainment :14029.8
Business :7491.12
Education :7003.98
Catalogs :4004.0
Medical :612.0


In [30]:
avg_reviews_google = avg_reviews(freq_category_google, reviews_count_category_google)
    
for row in avg_reviews_google:
    print(row[1], ':{:.7}'.format(row[0]))

COMMUNICATION :995608.5
SOCIAL :965831.0
GAME :683523.8
VIDEO_PLAYERS :425350.1
PHOTOGRAPHY :404081.4
TOOLS :305732.9
ENTERTAINMENT :301752.2
SHOPPING :223887.3
PERSONALIZATION :181122.3
WEATHER :171250.8
PRODUCTIVITY :160634.5
MAPS_AND_NAVIGATION :142860.0
TRAVEL_AND_LOCAL :129484.4
SPORTS :116938.6
FAMILY :113210.5
NEWS_AND_MAGAZINES :93088.03
BOOKS_AND_REFERENCE :87995.07
HEALTH_AND_FITNESS :78094.97
FOOD_AND_DRINK :57478.79
EDUCATION :56293.1
COMICS :42585.62
FINANCE :38535.9
LIFESTYLE :33921.82
HOUSE_AND_HOME :26435.47
ART_AND_DESIGN :24699.42
BUSINESS :24239.73
DATING :21953.27
PARENTING :16378.71
AUTO_AND_VEHICLES :14140.28
LIBRARIES_AND_DEMO :10925.81
BEAUTY :7476.226
MEDICAL :3730.153
EVENTS :2555.841


In [31]:
avg_inst_google = avg_reviews(freq_category_google, installs_count_category_google)
    
for row in avg_inst_google:
    print(row[1], ':{:.9}'.format(row[0]))

COMMUNICATION :38456119.2
VIDEO_PLAYERS :24727872.5
SOCIAL :23253652.1
PHOTOGRAPHY :17840110.4
PRODUCTIVITY :16787331.3
GAME :15588015.6
TRAVEL_AND_LOCAL :13984077.7
ENTERTAINMENT :11640705.9
TOOLS :10801391.3
NEWS_AND_MAGAZINES :9549178.47
BOOKS_AND_REFERENCE :8767811.89
SHOPPING :7036877.31
PERSONALIZATION :5201482.61
WEATHER :5074486.2
HEALTH_AND_FITNESS :4188821.99
MAPS_AND_NAVIGATION :4056941.77
FAMILY :3697848.17
SPORTS :3638640.14
ART_AND_DESIGN :1986335.09
FOOD_AND_DRINK :1924897.74
EDUCATION :1833495.15
BUSINESS :1712290.15
LIFESTYLE :1437816.27
FINANCE :1387692.48
HOUSE_AND_HOME :1331540.56
DATING :854028.83
COMICS :817657.273
AUTO_AND_VEHICLES :647317.817
LIBRARIES_AND_DEMO :638503.735
PARENTING :542603.621
BEAUTY :513151.887
EVENTS :253542.222
MEDICAL :120550.62


По среднему кол-ву отзывов и установок игры не лидируют и нам следует обратить внимание на другие жанры. При этом надо не забывать, что в некоторых категориях есть приложения-монстры, которые генерируют большую часть данных. Например, в **'SOCIAL'**.

In [47]:
for row in google_free_apps:
    if row[1] == 'SOCIAL' and row[5] == '100,000,000+':
        print(row[0], row[5])  

Tumblr 100,000,000+
Pinterest 100,000,000+
Badoo - Free Chat & Dating App 100,000,000+
Tango - Live Video Broadcast 100,000,000+
LinkedIn 100,000,000+
Tik Tok - including musical.ly 100,000,000+
BIGO LIVE - Live Stream 100,000,000+
VK 100,000,000+


In [45]:
for row in apple_free_apps:
    if row[11] == 'Social Networking' and float(row[5]) > 100000:
        print(row[1], row[5]) 

Facebook 2974676
Pinterest 1061624
Skype for iPhone 373519
Messenger 351466
Tumblr 334293
WhatsApp Messenger 287589
Kik 260965
ooVoo – Free Video Call, Text and Voice 177501
TextNow - Unlimited Text + Calls 164963
Viber Messenger – Text & Call 164249
Followers - Social Analytics For Instagram 112778


Это необходимо учесть и выбрать категорию, в которой наше приложение не останется незамеченным на фоне гигантов. Просмотрев несколько категорий, остановимся на приложениях для фотографий.

In [32]:
for row in google_free_apps:
    if row[1] == 'PHOTOGRAPHY':
        print(row[0], row[5])    

TouchNote: Cards & Gifts 1,000,000+
FreePrints – Free Photos Delivered 1,000,000+
Groovebook Photo Books & Gifts 500,000+
Moony Lab - Print Photos, Books & Magnets ™ 50,000+
LALALAB prints your photos, photobooks and magnets 1,000,000+
Snapfish 1,000,000+
Motorola Camera 50,000,000+
HD Camera - Best Cam with filters & panorama 5,000,000+
LightX Photo Editor & Photo Effects 10,000,000+
Sweet Snap - live filter, Selfie photo edit 10,000,000+
HD Camera - Quick Snap Photo & Video 1,000,000+
B612 - Beauty & Filter Camera 100,000,000+
Waterfall Photo Frames 1,000,000+
Photo frame 100,000+
Huji Cam 5,000,000+
Unicorn Photo 1,000,000+
HD Camera 5,000,000+
Makeup Editor -Beauty Photo Editor & Selfie Camera 1,000,000+
Makeup Photo Editor: Makeup Camera & Makeup Editor 1,000,000+
Moto Photo Editor 5,000,000+
InstaBeauty -Makeup Selfie Cam 50,000,000+
Garden Photo Frames - Garden Photo Editor 500,000+
Photo Frame 10,000,000+
Selfie Camera - Photo Editor & Filter & Sticker 50,000,000+
Sweet Snap Li

In [48]:
for row in apple_free_apps:
    if row[11] == 'Photo & Video':
        print(row[1], row[5]) 

Instagram 2161558
Snapchat 323905
YouTube - Watch Videos, Music, and Live Streams 278166
Pic Collage - Picture Editor & Photo Collage Maker 123433
Funimate video editor: add cool effects to videos 123268
musical.ly - your video social network 105429
Photo Collage Maker & Photo Editor - Live Collage 93781
Vine Camera 90355
Google Photos - unlimited photo and video storage 88742
Flipagram 79905
Mixgram - Picture Collage Maker - Pic Photo Editor 54282
Shutterfly: Prints, Photo Books, Cards Made Easy 51427
Pic Jointer – Photo Collage, Camera Effects Editor 51330
Color Pop Effects - Photo Editor & Picture Editing 45320
Photo Grid - photo collage maker & photo editor 40531
iSwap Faces LITE 39722
MOLDIV - Photo Editor, Collage & Beauty Camera 39501
Photo Editor by Aviary 39501
Photo Lab: Picture Editor, effects & fun face app 34585
Rookie Cam - Photo Editor & Filter Camera 33921
FotoRus -Camera & Photo Editor & Pic Collage Maker 32558
PicsArt Photo Studio: Collage Maker & Pic Editor 29078
Qui

В этой категории тоже есть сильные приложения, например **Instagram** или **YouTube**, но мы не собираемся с ними конкурировать. Вместе с ними в списках есть куча **редакторов фото**, которые нас и интересуют.

#### Заключение
---

В данном проекте мы проанализировали два набора данных с приложениями: из **Appstore** и **GooglePlay**, чтобы найти наиболее прибыльное приложение.

При этом мы ориентировались только на бесплатные приложения на английском языке, для чего изначальные наборы данных были очищены от всех нерелевантных записей.

В конце, основываясь на кол-ве установок и отзывов, а также на популярности самого жанра приложений, мы остановились на **редакторах фотографий**. Люди постоянно делают фотографи и постят их повсеместно. Мы же можем предложить приложение, которое поможет им улучщить их снимки, добавить какие-либо эффекты, создать коллажи  т.д. Оно обязательное займёт свою нишу в магазинах.